In [1]:
import sys
import os

# Add the pyCAFE directory to Python path
sys.path.append(os.path.join(os.getcwd(), 'pyCAFE_project'))

# Now import pycafe
import pyCAFE

# Verify it worked
print(pyCAFE.__file__)

/root/autodl-tmp/pyCAFE_project/pyCAFE/__init__.py


In [6]:
# See what's in the pycafe package
print(dir(pyCAFE))

['CUDA_AVAILABLE', '__all__', '__author__', '__builtins__', '__cached__', '__doc__', '__email__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'benchmark', 'benchmark_cpu_vs_gpu', 'clustering', 'core', 'cp', 'cuKMeans', 'extract_and_save_fullres_frames', 'extract_downsampled_frames_gpu', 'extract_frames_kmeans_cpu', 'extract_frames_kmeans_gpu', 'extract_specific_frames', 'extraction', 'get_video_info', 'perform_kmeans_gpu', 'pipelines', 'utils']


In [7]:
from pyCAFE import core, pipelines, clustering, extraction, utils

# or
import pyCAFE.core
import pyCAFE.pipelines
import pyCAFE.clustering

In [8]:
# See what functions/classes are in a module
import pyCAFE.core
print(dir(pyCAFE.core))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'extract_downsampled_frames_gpu', 'extract_frames_kmeans_cpu', 'extract_frames_kmeans_gpu', 'extract_specific_frames', 'get_video_info', 'perform_kmeans_gpu', 'time']


In [ ]:
import pyCAFE

# Basic usage
frames, timing = pyCAFE.extract_frames_kmeans_gpu(
    video_path="004-20221221.avi",
    output_dir="./frames",
    n_frames=50,
    step=5
)

print(f"Extracted {len(frames)} frames in {timing['total_time']:.2f}s")

In [ ]:
import pyCAFE

# Get video information
info = pyCAFE.get_video_info("004-20221221.avi")
print(f"Video: {info['nframes']} frames, {info['fps']:.2f} FPS")

# Advanced extraction with custom parameters
frames, timing = pyCAFE.extract_frames_kmeans_gpu(
    video_path="004-20221221.avi",
    output_dir="./frames",
    n_frames=100,
    start_time=0.1,      # Start at 10%
    end_time=0.9,        # End at 90%
    step=10,             # Sample every 10th frame
    resize_width=50,     # Larger thumbnails for clustering
    use_color=True,      # Use RGB (more accurate but slower)
    chunk_size=2500,     # Manual chunk size
    max_iter=200,        # More K-means iterations
    random_state=42      # Reproducible results
)

In [3]:
# Benchmark CPU vs GPU
results = pyCAFE.benchmark_cpu_vs_gpu(
    video_path="LL1-1_BalbcJ.mp4",
    n_frames=50,
    step=5
)
print(f"GPU is {results['speedup']:.2f}x faster")

🔥 BENCHMARK: CPU vs GPU

🚀 GPU Test...

VIDEO INFORMATION (GPU MODE)
Path: LL1-1_BalbcJ.mp4
Total frames: 99150
FPS: 30.00
Duration: 3305.00s
Resolution: 480x480

EXTRACTION PARAMETERS
Frame range: 0 - 99150
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False
Chunk size: auto

STEP 1: EXTRACTING FRAMES (GPU - NVIDIA DALI)

📹 Extracting 19830 downsampled frames on GPU...
   Range: 0-99150, step: 5
   Resize: 30, Color: False
   Auto chunk size: 3000
   Processing in 7 chunks
   (Prevents slowdown by rebuilding pipeline)



Chunk 1/7: 100%|██████████████████████████████████████████| 3000/3000 [00:01<00:00, 2167.09frames/s]


   ✅ Chunk 1/7 complete: 3000 frames



Chunk 2/7: 100%|██████████████████████████████████████████| 3000/3000 [00:01<00:00, 2560.08frames/s]


   ✅ Chunk 2/7 complete: 3000 frames



...

✅ Saved 50 frames to ./gpu_output
✅ Extracted full-res frames in 0.65s

✅ GPU EXTRACTION COMPLETE
Step 1 (Frame extraction):    35.86s ( 97.7%)
Step 2 (K-means):              0.19s (  0.5%)
Step 3 (Full-res extract):     0.65s (  1.8%)
----------------------------------------------------------------------
TOTAL TIME:                   36.69s
Extracted 50 frames
Output directory: ./gpu_output

💻 CPU Test...

VIDEO INFORMATION (CPU MODE)
Path: LL1-1_BalbcJ.mp4
Total frames: 99150
FPS: 30.00
Duration: 3305.00s
Resolution: 480x480

EXTRACTION PARAMETERS
Frame range: 0 - 99150
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False

STEP 1: EXTRACTING FRAMES (CPU - OpenCV)


Reading frames: 100%|██████████| 19830/19830 [01:58<00:00, 166.82it/s]


✅ Extracted 19830 frames in 118.95s
   Speed: 166.7 frames/s

STEP 2: K-MEANS CLUSTERING (CPU - scikit-learn)
Clustering data shape: (19830, 900)
Number of clusters: 50
✅ K-means converged in 24.87s
   Iterations: 81

STEP 3: EXTRACTING FULL-RESOLUTION FRAMES
Extracting 50 frames at full resolution...


Extracting frames: 100%|██████████| 50/50 [00:00<00:00, 76.48it/s]

✅ Saved 50 frames to ./cpu_output
✅ Extracted full-res frames in 0.68s

✅ CPU EXTRACTION COMPLETE
Step 1 (Frame extraction):   118.95s ( 82.3%)
Step 2 (K-means):             24.87s ( 17.2%)
Step 3 (Full-res extract):     0.68s (  0.5%)
----------------------------------------------------------------------
TOTAL TIME:                  144.50s
Extracted 50 frames
Output directory: ./cpu_output

📊 BENCHMARK RESULTS - CPU vs GPU COMPARISON

Step                                CPU (s)      GPU (s)      Speedup
----------------------------------------------------------------------
1. Frame Extraction                  118.95        35.86        3.32x
2. K-means Clustering                 24.87         0.19      134.28x
3. Full-res Extraction                 0.68         0.65        1.04x
----------------------------------------------------------------------
TOTAL                                144.50        36.69        3.94x

🎯 OVERALL SPEEDUP: 3.94x faster with GPU

✅ CPU extracted 50 frame

In [4]:
# Benchmark CPU vs GPU
results = pyCAFE.benchmark_cpu_vs_gpu(
    video_path="95-WT-male-56-20200615.avi",
    n_frames=50,
    step=5
)
print(f"GPU is {results['speedup']:.2f}x faster")

🔥 BENCHMARK: CPU vs GPU

🚀 GPU Test...

VIDEO INFORMATION (GPU MODE)
Path: 95-WT-male-56-20200615.avi
Total frames: 27000
FPS: 30.00
Duration: 900.00s
Resolution: 640x480

EXTRACTION PARAMETERS
Frame range: 0 - 27000
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False
Chunk size: auto

STEP 1: EXTRACTING FRAMES (GPU - NVIDIA DALI)

📹 Extracting 5400 downsampled frames on GPU...
   Range: 0-27000, step: 5
   Resize: 30, Color: False
   Auto chunk size: 2000
   Processing in 3 chunks
   (Prevents slowdown by rebuilding pipeline)



Chunk 1/3: 100%|██████████████████████████████████████████| 2000/2000 [00:01<00:00, 1606.18frames/s]


   ✅ Chunk 1/3 complete: 2000 frames



Chunk 2/3: 100%|██████████████████████████████████████████| 2000/2000 [00:01<00:00, 1657.76frames/s]


   ✅ Chunk 2/3 complete: 2000 frames



...

✅ Saved 50 frames to ./gpu_output
✅ Extracted full-res frames in 1.09s

✅ GPU EXTRACTION COMPLETE
Step 1 (Frame extraction):     8.32s ( 87.5%)
Step 2 (K-means):              0.09s (  1.0%)
Step 3 (Full-res extract):     1.09s ( 11.5%)
----------------------------------------------------------------------
TOTAL TIME:                    9.51s
Extracted 50 frames
Output directory: ./gpu_output

💻 CPU Test...

VIDEO INFORMATION (CPU MODE)
Path: 95-WT-male-56-20200615.avi
Total frames: 27000
FPS: 30.00
Duration: 900.00s
Resolution: 640x480

EXTRACTION PARAMETERS
Frame range: 0 - 27000
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False

STEP 1: EXTRACTING FRAMES (CPU - OpenCV)


Reading frames: 100%|██████████| 5400/5400 [01:00<00:00, 89.65it/s]


✅ Extracted 5400 frames in 60.25s
   Speed: 89.6 frames/s

STEP 2: K-MEANS CLUSTERING (CPU - scikit-learn)
Clustering data shape: (5400, 660)
Number of clusters: 50
✅ K-means converged in 5.35s
   Iterations: 48

STEP 3: EXTRACTING FULL-RESOLUTION FRAMES
Extracting 50 frames at full resolution...


Extracting frames: 100%|██████████| 50/50 [00:01<00:00, 46.34it/s]

✅ Saved 50 frames to ./cpu_output
✅ Extracted full-res frames in 1.09s

✅ CPU EXTRACTION COMPLETE
Step 1 (Frame extraction):    60.25s ( 90.3%)
Step 2 (K-means):              5.35s (  8.0%)
Step 3 (Full-res extract):     1.09s (  1.6%)
----------------------------------------------------------------------
TOTAL TIME:                   66.69s
Extracted 50 frames
Output directory: ./cpu_output

📊 BENCHMARK RESULTS - CPU vs GPU COMPARISON

Step                                CPU (s)      GPU (s)      Speedup
----------------------------------------------------------------------
1. Frame Extraction                   60.25         8.32        7.24x
2. K-means Clustering                  5.35         0.09       56.90x
3. Full-res Extraction                 1.09         1.09        0.99x
----------------------------------------------------------------------
TOTAL                                 66.69         9.51        7.01x

🎯 OVERALL SPEEDUP: 7.01x faster with GPU

✅ CPU extracted 50 frame

In [5]:
# Benchmark CPU vs GPU
results = pyCAFE.benchmark_cpu_vs_gpu(
    video_path="2018-4-30_NP_A11_F_N_S_25.mp4",
    n_frames=50,
    step=5
)
print(f"GPU is {results['speedup']:.2f}x faster")

🔥 BENCHMARK: CPU vs GPU

🚀 GPU Test...

VIDEO INFORMATION (GPU MODE)
Path: 2018-4-30_NP_A11_F_N_S_25.mp4
Total frames: 1981
FPS: 30.02
Duration: 66.00s
Resolution: 1280x720

EXTRACTION PARAMETERS
Frame range: 0 - 1981
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False
Chunk size: auto

STEP 1: EXTRACTING FRAMES (GPU - NVIDIA DALI)

📹 Extracting 397 downsampled frames on GPU...
   Range: 0-1981, step: 5
   Resize: 30, Color: False
   Auto chunk size: 397


Chunk 1/1: 100%|█████████████████████████████████████████████| 397/397 [00:00<00:00, 682.29frames/s]


   ✅ Chunk 1/1 complete: 397 frames

↗️  Moving data to GPU (CuPy array)
GPU memory used: 0.00 GB

✅ Extracted 397 frames
   Shape: (397, 30, 53)
✅ Extracted frames in 1.05s

STEP 2: K-MEANS CLUSTERING (GPU - cuML)
Clustering data shape: (397, 1590)
Number of clusters: 50
Max iterations: 100
Using cuML KMeans (GPU acceleration)
Fitting k-means on GPU...
✅ K-means converged
   Iterations: 101

Selecting representative frames...
⚠️  Cluster 0 is empty, skipping
⚠️  Cluster 1 is empty, skipping
⚠️  Cluster 2 is empty, skipping
⚠️  Cluster 3 is empty, skipping
⚠️  Cluster 4 is empty, skipping
⚠️  Cluster 5 is empty, skipping
⚠️  Cluster 7 is empty, skipping
⚠️  Cluster 8 is empty, skipping
⚠️  Cluster 10 is empty, skipping
⚠️  Cluster 12 is empty, skipping
⚠️  Cluster 13 is empty, skipping
⚠️  Cluster 14 is empty, skipping
⚠️  Cluster 15 is empty, skipping
⚠️  Cluster 16 is empty, skipping
⚠️  Cluster 17 is empty, skipping
⚠️  Cluster 19 is empty, skipping
⚠️  Cluster 20 is empty, skipping

Extracting frames: 100%|██████████| 7/7 [00:00<00:00,  8.32it/s]


✅ Saved 7 frames to ./gpu_output
✅ Extracted full-res frames in 0.87s

✅ GPU EXTRACTION COMPLETE
Step 1 (Frame extraction):     1.05s ( 52.1%)
Step 2 (K-means):              0.09s (  4.7%)
Step 3 (Full-res extract):     0.87s ( 43.2%)
----------------------------------------------------------------------
TOTAL TIME:                    2.01s
Extracted 7 frames
Output directory: ./gpu_output

💻 CPU Test...

VIDEO INFORMATION (CPU MODE)
Path: 2018-4-30_NP_A11_F_N_S_25.mp4
Total frames: 1981
FPS: 30.02
Duration: 66.00s
Resolution: 1280x720

EXTRACTION PARAMETERS
Frame range: 0 - 1981
Sampling step: 5
Frames to extract: 50
Clustering resize: 30px
Use color: False

STEP 1: EXTRACTING FRAMES (CPU - OpenCV)


Reading frames: 100%|██████████| 397/397 [00:24<00:00, 16.38it/s]


✅ Extracted 397 frames in 24.26s
   Speed: 16.4 frames/s

STEP 2: K-MEANS CLUSTERING (CPU - scikit-learn)
Clustering data shape: (397, 480)
Number of clusters: 50
✅ K-means converged in 0.39s
   Iterations: 9

STEP 3: EXTRACTING FULL-RESOLUTION FRAMES
Extracting 50 frames at full resolution...


Extracting frames: 100%|██████████| 50/50 [00:04<00:00, 10.39it/s]

✅ Saved 50 frames to ./cpu_output
✅ Extracted full-res frames in 4.84s

✅ CPU EXTRACTION COMPLETE
Step 1 (Frame extraction):    24.26s ( 82.3%)
Step 2 (K-means):              0.39s (  1.3%)
Step 3 (Full-res extract):     4.84s ( 16.4%)
----------------------------------------------------------------------
TOTAL TIME:                   29.48s
Extracted 50 frames
Output directory: ./cpu_output

📊 BENCHMARK RESULTS - CPU vs GPU COMPARISON

Step                                CPU (s)      GPU (s)      Speedup
----------------------------------------------------------------------
1. Frame Extraction                   24.26         1.05       23.15x
2. K-means Clustering                  0.39         0.09        4.10x
3. Full-res Extraction                 4.84         0.87        5.56x
----------------------------------------------------------------------
TOTAL                                 29.48         2.01       14.65x

🎯 OVERALL SPEEDUP: 14.65x faster with GPU

✅ CPU extracted 50 fram